In [90]:
import geopandas as gpd
import pandas as pd
from pandas.io.json import json_normalize
import urllib.request
import json
from geopandas import GeoDataFrame
from shapely.geometry import Point
from datetime import datetime, timedelta

In [37]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [91]:
def convert_coordinates(coordinates, srcEPSG, dstEPSG):
    """
    Converts points to coordinatesystem
    
    Input:
    coordinates: (list of) tuples
    """    
    from osgeo.osr import SpatialReference, CoordinateTransformation

    src = SpatialReference()
    dst = SpatialReference()
    
    src.ImportFromEPSG(srcEPSG)
    dst.ImportFromEPSG(dstEPSG)
    
    transform = CoordinateTransformation(src, dst)

    if not type(coordinates) is list:
        coordinates = [(coordinates)]
       
    if len(coordinates[0]) == 2:
        dimension = 2
    else:
        dimension = 3
       
    transformed = []
    for crd in coordinates:
        transformed.append((transform.TransformPoint(*crd)))
    
    if dimension == 2:
        transformed = [(i[0], i[1]) for i in transformed]
    return transformed

In [93]:
# set timedelta in epoch
min_timedelta = 0
time_epoch = int((datetime.now() - timedelta(minutes=min_timedelta)).timestamp())
print ((datetime.now() - timedelta(minutes=min_timedelta)).strftime('%c'))
print (time_epoch)

Fri Jun  2 17:18:37 2017
1496416717


In [105]:
# urls to retrieve AIS data from data providers
#url_marinetraffic  = 'https://www.marinetraffic.com/getData/get_data_json_4/z:5/X:7/Y:4/station:0'
minlat = str(52.316734)
maxlat = str(52.476281)
minlon = str(4.422955)
maxlon = str(5.059489)
timecode = str(time_epoch)
mmsi = str(244660778)
zoom = str(10)

url_vesselsonmap = 'http://www.myshiptracking.com/requests/vesselsonmap.php?type=json&minlat='+minlat+'&maxlat='+maxlat+'&minlon='+minlon+'&maxlon='+maxlon+'&zoom='+zoom+'&mmsi='+mmsi+'&timecode='+timecode
print (url_vesselsonmap)

http://www.myshiptracking.com/requests/vesselsonmap.php?type=json&minlat=52.316734&maxlat=52.476281&minlon=4.422955&maxlon=5.059489&zoom=10&mmsi=244660778&timecode=1496416717


In [95]:
# request the tracking url and decode as json and parse to pandas dataframe
r = urllib.request.urlopen(url_vesselsonmap)
data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
df = json_normalize(data[0]['DATA'])

In [96]:
# drop all the NaNs
# and convert strings to floats
df.dropna(inplace=True)
df.LNG = df.LNG.astype(float)
df.LAT = df.LAT.astype(float)

In [97]:
## convert LNG and LAT columns to geodataframe
# geometry = [Point(xy) for xy in zip(df.LNG.astype(float), df.LAT.astype(float))]
# df = df.drop(['LNG', 'LAT'], axis=1)
# crs = {'init': 'epsg:4326'}
# geo_df = GeoDataFrame(df, crs=crs, geometry=geometry)

In [98]:
df.head()

,ARV,COG,DEST,IMO,LAT,LNG,MMSI,NAME,R,S1,S2,S3,S4,SOG,T,TYPE,T_OLD
3,,0,,,52.59083,4.18889,992446026,WTG_41_PAWP,1496416721,56,56,56,56,0,1496416721,13,1496416721
5,,25,IJMUIDEN<>Q7,0,52.59178,4.23204,235094496,ADMIRAL P,1496416727,4,14,4,2,0.1,1496416727,4,1496416727
18,,511,,,52.61132,5.09455,244830228,,1496416721,,,,,4.8,1496416721,,1496416721
31,,511,,0,52.59738,5.37098,244670128,HERCULES,1496416725,32,10,2,5,0,1496416725,9,1496416725
33,6/1/2017 4:11:51 PM,511,ENKHUIZEN,0,52.6963,5.28746,244730367,DE STRIJD,1496416729,28,5,1,5,0,1496416729,6,1496416729


In [99]:
import folium

mapa = folium.Map([52.419052, 4.800132],
                  zoom_start=int(zoom),
                  tiles='cartodbpositron')

fg=folium.FeatureGroup(name="ships")
#points = folium.features.GeoJson(df)

for lat,lon,name,dest in zip(df['LAT'],df['LNG'],df['NAME'],df['DEST']):
    #print (lat,lon,name,dest)
    try:
        fg.add_child(folium.Marker(location=[lat,lon],popup=(folium.Popup('NAME: '+name+'  '+'DEST: '+dest))))
    except:
        continue

mapa.add_child(fg)
mapa.add_child(folium.LayerControl())

In [ ]:
# START CREATING THE NODE XML FILE

In [101]:
# lat/lon coordinates in 4326
in_points = [(52.379972,4.958865),(51.967049,5.320434),(52.466740,4.606085),(52.014135,5.110308),(52.036883,5.112706),(52.374937,4.948080),(52.420181,4.809713),(52.396220,4.813572),(52.521354,3.521670),(53.036403,4.142134),(52.075505,2.839227)]

# corresponding nodes
loc_codes = ['OSZ','PIZ','SIJ','PBS','LKD','GMC','HPT','HHN','NZW','NZN','NZZ']

# convert to rd coordinates (28992)
c = []
for b in in_points: c.append(b[::-1])
rd_coords = convert_coordinates(c, 4326, 28992)

In [102]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment

In [103]:
# CREATE XML NODES
nodes = Element('nodes')
for idx in range(len(loc_codes)):
    #print (idx)
    loc = loc_codes[idx]
    x   = str(rd_coords[idx][0])
    y   = str(rd_coords[idx][1]    )
    #print (x,y,loc) 
    child_1 = SubElement(nodes, 'node')
    child_1.set('id',loc)
    child_1.set('x',x)
    child_1.set('y',y)
print(prettify(nodes))

<?xml version="1.0" ?>
<nodes>
  <node id="OSZ" x="125834.83852422072" y="488097.9603182094"/>
  <node id="PIZ" x="150411.48229723892" y="442071.74998949823"/>
  <node id="SIJ" x="101919.03922584091" y="497952.3548312712"/>
  <node id="PBS" x="135991.09351610707" y="447344.51527389255"/>
  <node id="LKD" x="136165.27718886" y="449874.77603997145"/>
  <node id="GMC" x="125097.11133994945" y="487542.1432209603"/>
  <node id="HPT" x="115715.02955496815" y="492642.21309771115"/>
  <node id="HHN" x="115956.43341168977" y="489974.14888488414"/>
  <node id="NZW" x="28392.35379833178" y="505374.47147953056"/>
  <node id="NZN" x="71485.07769248312" y="561775.994354166"/>
  <node id="NZZ" x="-19654.88357901157" y="457201.441553026"/>
</nodes>



In [ ]:
# END CREATING THE NODE XML FILE

In [106]:
# REQUEST SINGLE SHIP INFORMATION

In [109]:
# urls to retrieve AIS data from data providers
mmsi = str(244770329)

url_vesseldetails = 'http://www.myshiptracking.com/requests/vesseldetails.php?type=json&mmsi='+mmsi
print (url_vesseldetails)

http://www.myshiptracking.com/requests/vesseldetails.php?type=json&mmsi=244770329


In [112]:
# request the tracking url and decode as json and parse to pandas dataframe
r = urllib.request.urlopen(url_vesseldetails)
data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
df = json_normalize(data)

In [114]:
df.T

,0
P,[]
TZ,
V.COG,511
V.DESTINATION,IJMUIDEN
V.DRAUGHT,0
V.ETA,
V.FLAG,Netherlands[NL]
V.LAT,52.4687
V.LDEST,IJMUIEN D
V.LDESTT,2017-05-30 09:45
